# Import the data




In [0]:
! pip install scikit-learn --upgrade # load newest version of sklearn

Requirement already up-to-date: scikit-learn in /usr/local/lib/python3.6/dist-packages (0.22.1)


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

data=pd.read_csv("worldhappiness2019_onenewvariable.csv")

data.head()

,Happiness_level,Country or region,GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,Terrorist_attacks
0,Average,Hong Kong,1.438,1.277,1.122,0.440,0.258,0.287,585.434783
1,Average,Moldova,0.685,1.328,0.739,0.245,0.181,0.000,2.000000
2,Average,North Macedonia,0.983,1.294,0.838,0.345,0.185,0.034,585.434783
3,Average,Northern Cyprus,1.263,1.252,1.042,0.417,0.191,0.162,585.434783
4,Average,Russia,1.183,1.452,0.726,0.334,0.082,0.031,559.000000


In [3]:
# Truncated and cleaned up region data to merge (Week 4 folder)
countrydata=pd.read_csv("newcountryvars.csv")

countrydata.head()

,country_name,country_code,country_code3,continent,region,population,population_below_poverty_line,hdi,life_expectancy,expected_years_of_schooling,mean_years_of_schooling,gni,kiva_country_name
0,India,IN,IND,Asia,Southern Asia,1339180127,21.9,0.623559,68.322,11.696590,6.298834,5663.474799,India
1,Nigeria,NG,NGA,Africa,Western Africa,190886311,70.0,0.527105,53.057,9.970482,6.000000,5442.901264,Nigeria
2,Mexico,MX,MEX,Americas,Central America,129163276,46.2,0.761683,76.972,13.299090,8.554985,16383.106680,Mexico
3,Pakistan,PK,PAK,Asia,Southern Asia,197015955,29.5,0.550354,66.365,8.106910,5.089460,5031.173074,Pakistan
4,Bangladesh,BD,BGD,Asia,Southern Asia,164669751,31.5,0.578824,71.985,10.178706,5.241577,3341.490722,Bangladesh


In [4]:
# Clean up final data

mergedata=pd.merge(data, countrydata, how='left', left_on='Country or region', right_on='country_name')
# Check for missing values (there won't be any given that I have already cleaned up the region data)
mergedata.loc[pd.isnull(mergedata).iloc[:,9]].to_csv("missing.csv",index=False)

mergedata.head()

,Happiness_level,Country or region,GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,Terrorist_attacks,country_name,country_code,country_code3,continent,region,population,population_below_poverty_line,hdi,life_expectancy,expected_years_of_schooling,mean_years_of_schooling,gni,kiva_country_name
0,Average,Hong Kong,1.438,1.277,1.122,0.440,0.258,0.287,585.434783,Hong Kong,HK,HKG,Asia,Eastern Asia,7364883.0,19.6,0.917157,84.163,15.67329,11.590666,54264.914780,Hong Kong
1,Average,Moldova,0.685,1.328,0.739,0.245,0.181,0.000,2.000000,Moldova,MD,MDA,Europe,Eastern Europe,4051212.0,20.8,0.699041,71.731,11.81358,11.915960,5026.259922,Moldova
2,Average,North Macedonia,0.983,1.294,0.838,0.345,0.185,0.034,585.434783,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Average,Northern Cyprus,1.263,1.252,1.042,0.417,0.191,0.162,585.434783,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Average,Russia,1.183,1.452,0.726,0.334,0.082,0.031,559.000000,Russia,RU,RUS,Europe,Eastern Europe,143989754.0,13.3,0.803908,70.264,14.95315,12.019990,23286.080650,Russia


In [10]:
# clean up final region data
X=mergedata.drop(['Happiness_level'],axis=1)
X=X.drop(['Country or region','country_name','country_code','country_code3','region','kiva_country_name'],axis=1)
X

,GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,Terrorist_attacks,continent,population,population_below_poverty_line,hdi,life_expectancy,expected_years_of_schooling,mean_years_of_schooling,gni
0,1.438,1.277,1.122,0.440,0.258,0.287,585.434783,Asia,7364883.0,19.6,0.917157,84.163,15.67329,11.590666,54264.914780
1,0.685,1.328,0.739,0.245,0.181,0.000,2.000000,Europe,4051212.0,20.8,0.699041,71.731,11.81358,11.915960,5026.259922
2,0.983,1.294,0.838,0.345,0.185,0.034,585.434783,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.263,1.252,1.042,0.417,0.191,0.162,585.434783,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.183,1.452,0.726,0.334,0.082,0.031,559.000000,Europe,143989754.0,13.3,0.803908,70.264,14.95315,12.019990,23286.080650
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,0.274,0.757,0.505,0.142,0.275,0.078,1419.095238,Africa,813912.0,44.8,0.497496,63.567,11.09573,4.831995,1335.004475
152,1.120,1.402,0.798,0.498,0.215,0.060,125.611111,Africa,1265138.0,8.0,0.781386,74.595,15.16978,9.094000,17947.963980
153,1.362,1.368,0.871,0.536,0.255,0.110,95.000000,Asia,1492584.0,NaN,0.823884,76.715,14.50000,9.419000,37236.363040
154,1.572,1.463,1.141,0.556,0.271,0.453,125.611111,Asia,5708844.0,NaN,0.924866,83.209,15.40000,11.572410,78162.323870


In [11]:
# Set up training and test data
from sklearn.model_selection import train_test_split

y=data['Happiness_level']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

print(X_train.shape)
print(y_train.shape)
print(X_train.columns.tolist())
print(X_train.dtypes)



(117, 15)
(117,)
['GDP per capita', 'Social support', 'Healthy life expectancy', 'Freedom to make life choices', 'Generosity', 'Perceptions of corruption', 'Terrorist_attacks', 'continent', 'population', 'population_below_poverty_line', 'hdi', 'life_expectancy', 'expected_years_of_schooling', 'mean_years_of_schooling', 'gni']
GDP per capita                   float64
Social support                   float64
Healthy life expectancy          float64
Freedom to make life choices     float64
Generosity                       float64
Perceptions of corruption        float64
Terrorist_attacks                float64
continent                         object
population                       float64
population_below_poverty_line    float64
hdi                              float64
life_expectancy                  float64
expected_years_of_schooling      float64
mean_years_of_schooling          float64
gni                              float64
dtype: object


In [0]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# We create the preprocessing pipelines for both numeric and categorical data.

numeric_features=X.columns.tolist()
numeric_features.remove('continent')

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_features = ['continent']

#Replacing missing values with Modal value and then one hot encoding.
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# final preprocessor object set up with ColumnTransformer

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])


#Fit your preprocessor object
prediction_input_preprocessor=preprocessor.fit(X_train) 

import pickle
pickle.dump(prediction_input_preprocessor, open( "preprocessor.pkl", "wb" ) )

In [17]:
# Model with best L2 regularization run for three times the epochs
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout,BatchNormalization
from keras.optimizers import SGD
from keras.regularizers import l1
from keras.regularizers import l2
from keras.regularizers import l1_l2
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras import initializers

opt = SGD(lr=0.01)

model = Sequential()
model.add(Dense(150, input_dim=19, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
model.add(Dropout(.3))
model.add(Dense(150, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
model.add(Dropout(.3))
model.add(Dense(150, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
model.add(Dropout(.3))
model.add(Dense(5, activation='softmax')) 
                                            
# Compile model
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

# save best model given maximum val_accuracy, stop early if loss does not improve after 200 further iterations beyond best loss
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=500)
mc = ModelCheckpoint('best_model.h5',monitor='val_acc', mode='max', verbose=1, save_best_only=True) # evaluating val_loss minimization

# Fitting the NN to the Training set
model.fit(prediction_input_preprocessor.transform(X_train), pd.get_dummies(y_train), 
          validation_data=(prediction_input_preprocessor.transform(X_test), pd.get_dummies(y_test)), epochs=1000, verbose=1, callbacks=[es,mc])


Train on 117 samples, validate on 39 samples
Epoch 1/1000
117/117 [==============================] - 1s 7ms/step - loss: 5.0361 - acc: 0.1966 - val_loss: 4.9499 - val_acc: 0.3590

Epoch 00001: val_acc improved from -inf to 0.35897, saving model to best_model.h5
Epoch 2/1000
117/117 [==============================] - 0s 201us/step - loss: 4.9942 - acc: 0.2051 - val_loss: 4.9298 - val_acc: 0.3846

Epoch 00002: val_acc improved from 0.35897 to 0.38462, saving model to best_model.h5
Epoch 3/1000
117/117 [==============================] - 0s 148us/step - loss: 4.9949 - acc: 0.1966 - val_loss: 4.9087 - val_acc: 0.4872

Epoch 00003: val_acc improved from 0.38462 to 0.48718, saving model to best_model.h5
Epoch 4/1000
117/117 [==============================] - 0s 142us/step - loss: 4.9591 - acc: 0.2564 - val_loss: 4.8895 - val_acc: 0.5128

Epoch 00004: val_acc improved from 0.48718 to 0.51282, saving model to best_model.h5
Epoch 5/1000
117/117 [==============================] - 0s 141us/step - 

In [9]:
from keras.models import load_model

model = load_model('best_model.h5')

#Now we have automated model building such that we can choose the best model evaluated on test data 
#throughout the model building process!


# using predict_classes() for multi-class data to return predicted class index.

prediction_index=model.predict_classes(prediction_input_preprocessor.transform(X_test))

#Now lets run some code to get keras to return the label rather than the index...

# get labels from one hot encoded y_train data
labels=pd.get_dummies(y_train).columns

# Iterate through all predicted indices using map method

predicted_labels=list(map(lambda x: labels[x], prediction_index))


# load model_eval_metrics() function into our session to calculate metrics

import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
import pandas as pd
from math import sqrt

def model_eval_metrics(y_true, y_pred,classification="TRUE"):
     if classification=="TRUE":
        accuracy_eval = accuracy_score(y_true, y_pred)
        f1_score_eval = f1_score(y_true, y_pred,average="macro",zero_division=0)
        precision_eval = precision_score(y_true, y_pred,average="macro",zero_division=0)
        recall_eval = recall_score(y_true, y_pred,average="macro",zero_division=0)
        mse_eval = 0
        rmse_eval = 0
        mae_eval = 0
        r2_eval = 0
        metricdata = {'accuracy': [accuracy_eval], 'f1_score': [f1_score_eval], 'precision': [precision_eval], 'recall': [recall_eval], 'mse': [mse_eval], 'rmse': [rmse_eval], 'mae': [mae_eval], 'r2': [r2_eval]}
        finalmetricdata = pd.DataFrame.from_dict(metricdata)
     else:
        accuracy_eval = 0
        f1_score_eval = 0
        precision_eval = 0
        recall_eval = 0
        mse_eval = mean_squared_error(y_true, y_pred)
        rmse_eval = sqrt(mean_squared_error(y_true, y_pred))
        mae_eval = mean_absolute_error(y_true, y_pred)
        r2_eval = r2_score(y_true, y_pred)
        metricdata = {'accuracy': [accuracy_eval], 'f1_score': [f1_score_eval], 'precision': [precision_eval], 'recall': [recall_eval], 'mse': [mse_eval], 'rmse': [rmse_eval], 'mae': [mae_eval], 'r2': [r2_eval]}
        finalmetricdata = pd.DataFrame.from_dict(metricdata)
     return finalmetricdata

model_eval_metrics( y_test,predicted_labels,classification="TRUE")


# add metrics to submittable object
modelevalobject=model_eval_metrics( y_test,predicted_labels,classification="TRUE")

modelevalobject

,accuracy,f1_score,precision,recall,mse,rmse,mae,r2
0,0.564103,0.545015,0.552273,0.579444,0,0,0,0
